# PyTorch Neural Network Classification Models

This notebook contains two separate PyTorch neural network classification models:
1. **Celebrity Dataset (CelebA)**
2. **Handwritten Digit Images (MNIST)**

Both models will be trained, validated, and tested using their respective datasets.


In [17]:
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision
import torchvision.transforms as transforms
from torch.utils.data import DataLoader
import torch.nn.functional as F 

import matplotlib.pyplot as plt
import numpy as np


In [18]:
import torch

# Check if CUDA is available
cuda_available = torch.cuda.is_available()
print(f"CUDA available: {cuda_available}")

# Check the number of GPUs available
gpu_count = torch.cuda.device_count()
print(f"Number of GPUs available: {gpu_count}")

# Get the name of the GPU
if cuda_available:
    gpu_name = torch.cuda.get_device_name(0)
    print(f"GPU Name: {gpu_name}")

CUDA available: True
Number of GPUs available: 1
GPU Name: NVIDIA GeForce RTX 4090


In [19]:
# Preparing the Datasets

# Transformation for both datasets
transform = transforms.Compose([
    transforms.Resize((64, 64)),  # Resize for CelebA compatibility
    transforms.ToTensor(),
    transforms.Normalize((0.5,), (0.5,))
])

# CelebA Dataset
celeba_dataset = torchvision.datasets.CelebA(root='./data', split='train', download=True, transform=transform)
celeba_loader = DataLoader(celeba_dataset, batch_size=32, shuffle=True)

# MNIST Dataset
mnist_dataset = torchvision.datasets.MNIST(root='./data', train=True, download=True, transform=transforms.ToTensor())
mnist_loader = DataLoader(mnist_dataset, batch_size=32, shuffle=True)

# For testing and validation
celeba_test_dataset = torchvision.datasets.CelebA(root='./data', split='test', download=True, transform=transform)
celeba_test_loader = DataLoader(celeba_test_dataset, batch_size=32, shuffle=False)

mnist_test_dataset = torchvision.datasets.MNIST(root='./data', train=False, download=True, transform=transforms.ToTensor())
mnist_test_loader = DataLoader(mnist_test_dataset, batch_size=32, shuffle=False)


Files already downloaded and verified
Files already downloaded and verified


In [26]:
# Defining the Neural Network for CelebA (CNN)

# Modify the last layer to output 40 values instead of 1
class CelebACNN(nn.Module):
    def __init__(self):
        super(CelebACNN, self).__init__()
        self.conv1 = nn.Conv2d(3, 16, kernel_size=3, stride=1, padding=1)
        self.conv2 = nn.Conv2d(16, 32, kernel_size=3, stride=1, padding=1)
        self.conv3 = nn.Conv2d(32, 64, kernel_size=3, stride=1, padding=1)
        self.pool = nn.MaxPool2d(kernel_size=2, stride=2, padding=0)
        self.fc1 = nn.Linear(64 * 8 * 8, 512)
        self.fc2 = nn.Linear(512, 40)  # Output 40 labels
        self.sigmoid = nn.Sigmoid()
        
    def forward(self, x):
        x = self.pool(F.relu(self.conv1(x)))
        x = self.pool(F.relu(self.conv2(x)))
        x = self.pool(F.relu(self.conv3(x)))
        x = x.view(-1, 64 * 8 * 8)
        x = F.relu(self.fc1(x))
        x = self.fc2(x)
        x = self.sigmoid(x)
        return x

# Initialize the model, loss function, and optimizer
celeba_model = CelebACNN()
criterion = nn.BCELoss()
optimizer = optim.Adam(celeba_model.parameters(), lr=0.001)


In [28]:
# Training the CelebA Model

epochs = 5
celeba_model = CelebACNN().cuda()
criterion = nn.BCELoss()
optimizer = optim.Adam(celeba_model.parameters(), lr=0.001)
celeba_model.train()

for epoch in range(epochs):
    running_loss = 0.0
    for i, data in enumerate(celeba_loader, 0):
        inputs, labels = data
        labels = labels.float().cuda()  # Ensure labels are on GPU and correctly shaped
        optimizer.zero_grad()
        outputs = celeba_model(inputs.cuda())
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        running_loss += loss.item()
        if i % 100 == 99:  # print every 100 mini-batches
            print(f'Epoch [{epoch + 1}/{epochs}], Step [{i + 1}/{len(celeba_loader)}], Loss: {running_loss / 100:.4f}')
            running_loss = 0.0

print('Finished Training CelebA Model')

Epoch [1/5], Step [100/5087], Loss: 0.4287
Epoch [1/5], Step [200/5087], Loss: 0.3740
Epoch [1/5], Step [300/5087], Loss: 0.3396
Epoch [1/5], Step [400/5087], Loss: 0.3164
Epoch [1/5], Step [500/5087], Loss: 0.3061
Epoch [1/5], Step [600/5087], Loss: 0.2948
Epoch [1/5], Step [700/5087], Loss: 0.2864
Epoch [1/5], Step [800/5087], Loss: 0.2837
Epoch [1/5], Step [900/5087], Loss: 0.2774
Epoch [1/5], Step [1000/5087], Loss: 0.2766
Epoch [1/5], Step [1100/5087], Loss: 0.2713
Epoch [1/5], Step [1200/5087], Loss: 0.2678
Epoch [1/5], Step [1300/5087], Loss: 0.2672
Epoch [1/5], Step [1400/5087], Loss: 0.2656
Epoch [1/5], Step [1500/5087], Loss: 0.2620
Epoch [1/5], Step [1600/5087], Loss: 0.2626
Epoch [1/5], Step [1700/5087], Loss: 0.2596
Epoch [1/5], Step [1800/5087], Loss: 0.2571
Epoch [1/5], Step [1900/5087], Loss: 0.2569
Epoch [1/5], Step [2000/5087], Loss: 0.2520
Epoch [1/5], Step [2100/5087], Loss: 0.2573
Epoch [1/5], Step [2200/5087], Loss: 0.2547
Epoch [1/5], Step [2300/5087], Loss: 0.25

In [29]:
# Defining the Neural Network for MNIST (CNN)

class MNISTCNN(nn.Module):
    def __init__(self):
        super(MNISTCNN, self).__init__()
        self.conv1 = nn.Conv2d(1, 16, kernel_size=3, stride=1, padding=1)
        self.conv2 = nn.Conv2d(16, 32, kernel_size=3, stride=1, padding=1)
        self.pool = nn.MaxPool2d(kernel_size=2, stride=2, padding=0)
        self.fc1 = nn.Linear(32 * 7 * 7, 128)
        self.fc2 = nn.Linear(128, 10)
        
    def forward(self, x):
        x = self.pool(F.relu(self.conv1(x)))
        x = self.pool(F.relu(self.conv2(x)))
        x = x.view(-1, 32 * 7 * 7)
        x = F.relu(self.fc1(x))
        x = self.fc2(x)
        return x

# Initialize the model, loss function, and optimizer
mnist_model = MNISTCNN()
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(mnist_model.parameters(), lr=0.001)


In [30]:
# Training the MNIST Model

epochs = 5
mnist_model.train()

for epoch in range(epochs):
    running_loss = 0.0
    for i, data in enumerate(mnist_loader, 0):
        inputs, labels = data
        optimizer.zero_grad()
        outputs = mnist_model(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        running_loss += loss.item()
        if i % 100 == 99:  # print every 100 mini-batches
            print(f'Epoch [{epoch + 1}/{epochs}], Step [{i + 1}/{len(mnist_loader)}], Loss: {running_loss / 100:.4f}')
            running_loss = 0.0

print('Finished Training MNIST Model')


Epoch [1/5], Step [100/1875], Loss: 1.1105
Epoch [1/5], Step [200/1875], Loss: 0.3680
Epoch [1/5], Step [300/1875], Loss: 0.2389
Epoch [1/5], Step [400/1875], Loss: 0.1657
Epoch [1/5], Step [500/1875], Loss: 0.1576
Epoch [1/5], Step [600/1875], Loss: 0.1308
Epoch [1/5], Step [700/1875], Loss: 0.1375
Epoch [1/5], Step [800/1875], Loss: 0.1038
Epoch [1/5], Step [900/1875], Loss: 0.0906
Epoch [1/5], Step [1000/1875], Loss: 0.0938
Epoch [1/5], Step [1100/1875], Loss: 0.0942
Epoch [1/5], Step [1200/1875], Loss: 0.0788
Epoch [1/5], Step [1300/1875], Loss: 0.0907
Epoch [1/5], Step [1400/1875], Loss: 0.0849
Epoch [1/5], Step [1500/1875], Loss: 0.0779
Epoch [1/5], Step [1600/1875], Loss: 0.0902
Epoch [1/5], Step [1700/1875], Loss: 0.0760
Epoch [1/5], Step [1800/1875], Loss: 0.0714
Epoch [2/5], Step [100/1875], Loss: 0.0639
Epoch [2/5], Step [200/1875], Loss: 0.0568
Epoch [2/5], Step [300/1875], Loss: 0.0600
Epoch [2/5], Step [400/1875], Loss: 0.0534
Epoch [2/5], Step [500/1875], Loss: 0.0484
Ep

In [35]:
def evaluate_model(model, test_loader, dataset_name, multi_label=False, threshold=0.5):
    correct = 0
    total = 0
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    model.to(device)
    model.eval()
    with torch.no_grad():
        for data in test_loader:
            images, labels = data
            images, labels = images.to(device), labels.to(device).float()
            outputs = model(images)
            
            if multi_label:
                # Apply threshold to outputs to get binary predictions for multi-label classification
                predicted = (outputs > threshold).float()
                # Compare predictions with actual labels
                correct += (predicted == labels).sum().item()
                total += labels.numel()  # Total number of labels (e.g., 40 per image)
            else:
                # Single-label classification (e.g., MNIST)
                _, predicted = torch.max(outputs, 1)
                correct += (predicted == labels).sum().item()
                total += labels.size(0)  # Total number of images

    accuracy = 100 * correct / total
    print(f'Accuracy of the model on the {dataset_name} test images: {accuracy:.2f}%')

# Evaluate CelebA model (multi-label classification)
evaluate_model(celeba_model, celeba_test_loader, 'CelebA', multi_label=True)

# Evaluate MNIST model (single-label classification)
evaluate_model(mnist_model, mnist_test_loader, 'MNIST', multi_label=False)

Accuracy of the model on the CelebA test images: 89.75%
Accuracy of the model on the MNIST test images: 98.84%
